In [1]:
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import joblib
from datetime import datetime, timedelta
from pathlib import Path
import glob
import pickle
from scipy import signal
import os


In [3]:
# WESAD dataset loader

class WesadDataLoader:
    
    @staticmethod
    def calculateHrv(rrIntervals):
        if len(rrIntervals) < 2:
            return None, None, None, None
        
        rrIntervals = rrIntervals[(rrIntervals > 300) & (rrIntervals < 2000)]
        
        if len(rrIntervals) < 2:
            return None, None, None, None
        
        sdnn = np.std(rrIntervals)
        diffRr = np.diff(rrIntervals)
        rmssd = np.sqrt(np.mean(diffRr ** 2))
        meanHr = 60000 / np.mean(rrIntervals)  # heart rate
        pnn50 = np.sum(np.abs(diffRr) > 50) / len(diffRr) * 100  # stress indicator
        
        return sdnn, rmssd, meanHr, pnn50
    
    @staticmethod
    def getRrFromEcg(ecgSignal, fs=700):
        nyq = fs / 2
        low = 0.5 / nyq
        high = 40 / nyq
        b, a = signal.butter(4, [low, high], btype='band')
        ecgFilt = signal.filtfilt(b, a, ecgSignal)
        
        ecgNorm = (ecgFilt - np.mean(ecgFilt)) / np.std(ecgFilt)
        
        minDist = int(0.4 * fs)
        thresh = np.mean(ecgNorm) + 0.5 * np.std(ecgNorm)
        peaks = signal.find_peaks(ecgNorm, height=thresh, distance=minDist)[0]
        
        if len(peaks) < 2:
            return None
        
        rr = np.diff(peaks) / fs * 1000
        
        return rr
    
    @staticmethod
    def loadWesadSubject(subjectPath):
        with open(subjectPath, 'rb') as f:
            data = pickle.load(f, encoding='latin1')
        
        if 'signal' not in data or 'chest' not in data['signal']:
            return None
        
        chest = data['signal']['chest']
        labels = data['label']
        
        if isinstance(chest, dict):
            ecg = chest['ECG'].flatten() if 'ECG' in chest else None
        else:
            ecg = chest[:, 0] if len(chest.shape) > 1 else chest
            
        if ecg is None:
            return None
        
        fs = 700
        
        return ecg, labels, fs
    
    @staticmethod
    def processWesadData(wesadDirectory):
        subjectFiles = glob.glob(f"{wesadDirectory}/**/S*.pkl", recursive=True)
        
        print(f"\nFound {len(subjectFiles)} WESAD subjects")
        
        if len(subjectFiles) == 0:
            return None
        
        features = []
        
        for subjFile in subjectFiles:
            subj = Path(subjFile).stem
            print(f"Processing {subj}...")
            
            result = WesadDataLoader.loadWesadSubject(subjFile)
            if result is None:
                continue
            
            ecg, labels, fs = result
            
            winSize = 60 * fs
            step = winSize // 2
            
            for i in range((len(ecg) - winSize) // step + 1):
                start = i * step
                end = start + winSize
                
                if end > len(ecg):
                    break
                
                ecgWin = ecg[start:end]
                labelWin = labels[start:end]
                
                vals, counts = np.unique(labelWin, return_counts=True)
                label = vals[np.argmax(counts)]
                
                if label == 0:
                    continue
                elif label == 1:
                    stress = 0
                elif label == 2:
                    stress = 2
                elif label == 3:
                    stress = 1
                elif label == 4:
                    stress = 0
                else:
                    continue
                
                rr = WesadDataLoader.getRrFromEcg(ecgWin, fs)
                
                if rr is None or len(rr) < 10:
                    continue
                
                sdnn, rmssd, meanHr, pnn50 = WesadDataLoader.calculateHrv(rr)
                
                if sdnn is None:
                    continue
                
                features.append({
                    'subject': subj,
                    'sdnn': sdnn,
                    'rmssd': rmssd,
                    'meanHr': meanHr,
                    'pnn50': pnn50,
                    'stressClass': stress,
                    'source': 'WESAD'
                })
        
        df = pd.DataFrame(features)
        
        print(f"\nWESAD data:")
        print(f"  Total samples: {len(df)}")
        print(f"  Low: {(df['stressClass'] == 0).sum()}")
        print(f"  Moderate: {(df['stressClass'] == 1).sum()}")
        print(f"  High: {(df['stressClass'] == 2).sum()}")
        print(f"  SDNN range: {df['sdnn'].min():.1f} - {df['sdnn'].max():.1f}")
        print(f"  RMSSD range: {df['rmssd'].min():.1f} - {df['rmssd'].max():.1f}")
        
        return df


In [5]:
# Samsung health data loader

class SamsungDataLoader:
    
    @staticmethod
    def loadAllSamsungFiles(dataDirectory):
        hrvFiles = glob.glob(dataDirectory + '/com.samsung.health.hrv/**/*.json', recursive=True)
        stressFiles = glob.glob(dataDirectory + '/com.samsung.shealth.stress/**/*.json', recursive=True)
        
        print(f"Found:")
        print(f"  Stress files: {len(stressFiles)}")
        print(f"  HRV files: {len(hrvFiles)}")
        
        stressData = []
        for file in stressFiles:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    stressData.extend(data)
                else:
                    stressData.append(data)
        
        hrvData = []
        for file in hrvFiles:
            if 'stress' in file.lower() or 'heart_rate' in file.lower():
                continue
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    hrvData.extend(data)
                else:
                    hrvData.append(data)
        
        return stressData, hrvData
    
    @staticmethod
    def processStressData(stressData):
        df = pd.DataFrame(stressData)
        df['timestamp'] = pd.to_datetime(df['start_time'], unit='ms')
        
        df['stressNormalized'] = df['score'] / df['score'].max()
        
        lowThresh = df['score'].quantile(0.33)
        highThresh = df['score'].quantile(0.66)
        
        df['stressClass'] = 1
        df.loc[df['score'] <= lowThresh, 'stressClass'] = 0
        df.loc[df['score'] > highThresh, 'stressClass'] = 2
        
        print(f"Stress data:")
        print(f"  Samples: {len(df)}")
        print(f"  Score range: {df['score'].min():.1f} - {df['score'].max():.1f}")
        print(f"  Low stress: {(df['stressClass'] == 0).sum()}")
        print(f"  Moderate stress: {(df['stressClass'] == 1).sum()}")
        print(f"  High stress: {(df['stressClass'] == 2).sum()}")
        
        return df[['timestamp', 'score', 'stressClass', 'stressNormalized']]
    
    @staticmethod
    def processHrvData(hrvData):
        df = pd.DataFrame(hrvData)
        df['timestamp'] = pd.to_datetime(df['start_time'], unit='ms')
        
        if 'sdnn' not in df.columns:
            df['sdnn'] = np.nan
        if 'rmssd' not in df.columns:
            df['rmssd'] = np.nan
        
        print(f"HRV data:")
        print(f"  Samples: {len(df)}")
        print(f"  SDNN range: {df['sdnn'].min():.1f} - {df['sdnn'].max():.1f}")
        print(f"  RMSSD range: {df['rmssd'].min():.1f} - {df['rmssd'].max():.1f}")
        
        return df[['timestamp', 'sdnn', 'rmssd']]
    
    @staticmethod
    def mergeStressAndHrv(dfStress, dfHrv):
        dfStress['timestampRounded'] = dfStress['timestamp'].dt.round('1min')
        dfHrv['timestampRounded'] = dfHrv['timestamp'].dt.round('1min')
        
        merged = pd.merge_asof(
            dfHrv.sort_values('timestamp'),
            dfStress.sort_values('timestamp'),
            on='timestamp',
            direction='nearest',
            tolerance=pd.Timedelta('1hr')
        )
        
        merged = merged.dropna(subset=['stressClass'])
        merged = merged.dropna(subset=['sdnn', 'rmssd'])
        
        print(f"Merged data:")
        print(f"  Matched samples: {len(merged)}")
        print(f"  Time span: {merged['timestamp'].min()} to {merged['timestamp'].max()}")
        print(f"  Duration: {(merged['timestamp'].max() - merged['timestamp'].min()).days} days")
        
        return merged[['timestamp', 'sdnn', 'rmssd', 'stressClass', 'score']]




In [7]:
# model training class

class StressClassifier:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = None
        self.features = ['sdnn', 'rmssd', 'meanHr', 'pnn50']  # add new features
    
    def train(self, X, y, testSize=0.2):
        xTrain, xTest, yTrain, yTest = train_test_split(
            X, y, test_size=testSize, random_state=42, stratify=y
        )
        
        xTrainScaled = self.scaler.fit_transform(xTrain)
        xTestScaled = self.scaler.transform(xTest)
        
        # calculate sample weights
        #from sklearn.utils.class_weight import compute_sample_weight
        #sampleWeights = compute_sample_weight('balanced', yTrain)
        from sklearn.utils.class_weight import compute_class_weight
        classWeights = compute_class_weight('balanced', classes=np.unique(yTrain), y=yTrain)
        classWeightDict = {0: classWeights[0], 1: classWeights[1]*2, 2: classWeights[2]}
        sampleWeights = np.array([classWeightDict[y] for y in yTrain])
        self.model = xgb.XGBClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            num_class=3
        )
        
        # add sample_weight here
        self.model.fit(xTrainScaled, yTrain, sample_weight=sampleWeights, verbose=False)

        
        yPred = self.model.predict(xTestScaled)
        yProba = self.model.predict_proba(xTestScaled)
        
        print("\nEVALUATION")
        print("="*50)
        
        labels = ['Low', 'Moderate', 'High']
        print(classification_report(yTest, yPred, target_names=labels))
        
        print(f"\nConfusion Matrix:")
        print(confusion_matrix(yTest, yPred))
        
        auc = roc_auc_score(yTest, yProba, multi_class='ovr', average='macro')
        print(f"\nROC-AUC: {auc:.3f}")
        
        print("\nFeature importance:")
        for f, imp in zip(self.features, self.model.feature_importances_):
            print(f"  {f}: {imp:.3f}")
        
        cm = confusion_matrix(yTest, yPred)
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, cmap='Blues')
        plt.colorbar()
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.xticks([0,1,2], labels)
        plt.yticks([0,1,2], labels)
        
        for i in range(3):
            for j in range(3):
                plt.text(j, i, cm[i,j], ha='center', va='center')
        
        plt.savefig('confusion.png', dpi=150)
        plt.close()
        print("\nSaved confusion.png")
        
        return self.model
    
    def predict(self, xNew):
        xScaled = self.scaler.transform(xNew)
        preds = self.model.predict(xScaled)
        proba = self.model.predict_proba(xScaled)
        return preds, proba
    
    def saveModel(self, modelFile='stress_model.pkl', scalerFile='scaler.pkl'):
        joblib.dump(self.model, modelFile)
        joblib.dump(self.scaler, scalerFile)
        print(f"\nSaved {modelFile} and {scalerFile}")


In [8]:
# production predictor for real-time use

class StressPredictor:
    
    def __init__(self, modelFile='stress_model.pkl', scalerFile='scaler.pkl', db=None):
        self.model = joblib.load(modelFile)
        self.scaler = joblib.load(scalerFile)
        self.db = db
        
        self.minConfidence = 0.60
        self.minSdnn = 10
        self.maxSdnn = 200
        self.minRmssd = 5
        self.maxRmssd = 150

                # demo user ID
        self.DEMO_USER_ID = "QOyZROlPzUf25tKPvv0FWnd3NZw2"
        self.DEMO_EMAIL = "test@email.com"
        
        print("Model loaded")
    
    def predictSingle(self, sdnn, rmssd, meanHr, pnn50, showDetails=True):
        if not (self.minSdnn <= sdnn <= self.maxSdnn):
            return {'error': f'Invalid SDNN: {sdnn:.2f}'}
        if not (self.minRmssd <= rmssd <= self.maxRmssd):
            return {'error': f'Invalid RMSSD: {rmssd:.2f}'}
        
        X = np.array([[sdnn, rmssd, meanHr, pnn50]])
        xScaled = self.scaler.transform(X)
        
        pred = self.model.predict(xScaled)[0]
        proba = self.model.predict_proba(xScaled)[0]
        
        stressLabels = {0: 'LOW', 1: 'MODERATE', 2: 'HIGH'}
        confidence = proba.max()
        
        result = {
            'stressLevel': int(pred),
            'stressLabel': stressLabels[pred],
            'confidence': float(confidence),
            'showToUser': confidence >= self.minConfidence,
            'probabilities': {
                'low': float(proba[0]),
                'moderate': float(proba[1]),
                'high': float(proba[2])
            }
        }
        
        if showDetails:
            print(f"\nInput: SDNN={sdnn:.2f}, RMSSD={rmssd:.2f}, MeanHR={meanHr:.2f}, PNN50={pnn50:.2f}")
            print(f"Predicted: {result['stressLabel']} (confidence: {confidence:.1%})")
            print(f"Show to user: {'YES' if result['showToUser'] else 'NO'}")
        
        return result
    
    def fetchAndPredictFromFirebase(self, hours=1):
        if self.db is None:
            print("No Firebase connection")
            return []
        
        cutoff = datetime.now() - timedelta(hours=hours)
        
        docs = self.db.collection('heart_data')\
                      .where('timestamp', '>=', cutoff)\
                      .order_by('timestamp')\
                      .stream()
        
        results = []
        for doc in docs:
            d = doc.to_dict()
            ibiList = d.get('ibi', [])
            
            if len(ibiList) < 50:
                continue
            
            ibi = np.array([x for x in ibiList if x > 0])
            sdnn = np.std(ibi)
            rmssd = np.sqrt(np.mean(np.diff(ibi) ** 2))
            
            pred = self.predictSingle(sdnn, rmssd, showDetails=False)
            
            if 'error' not in pred:
                pred['timestamp'] = d['timestamp']
                results.append(pred)
        
        return results

In [8]:
# main script

if __name__ == '__main__':
    print("Stress Classification - 3 Classes")
    print("="*50)
    
    samsungDir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\jsons'
    wesadDir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\WESAD'
    
    useSamsung = False
    useWesad = True
    
    allData = []
    
    if useSamsung:
        print("\nLoading Samsung data...")
        loader = SamsungDataLoader()
        stressData, hrvData = loader.loadAllSamsungFiles(samsungDir)
        
        if stressData and hrvData:
            dfStress = loader.processStressData(stressData)
            dfHrv = loader.processHrvData(hrvData)
            dfSamsung = loader.mergeStressAndHrv(dfStress, dfHrv)
            
            dfSamsung['source'] = 'Samsung'
            allData.append(dfSamsung[['sdnn', 'rmssd', 'stressClass', 'source']])
            print(f"Added {len(dfSamsung)} Samsung samples")
    
    if useWesad:
        print("\nLoading WESAD data...")
        wesadLoader = WesadDataLoader()
        dfWesad = wesadLoader.processWesadData(wesadDir)
        
        if dfWesad is not None and len(dfWesad) > 0:
            allData.append(dfWesad[['sdnn', 'rmssd', 'meanHr', 'pnn50', 'stressClass', 'source']])    
    print("\nCombining datasets...")
    df = pd.concat(allData, ignore_index=True)
    
    print(f"\nTotal samples: {len(df)}")
    for src in df['source'].unique():
        srcDf = df[df['source'] == src]
        print(f"\n{src}:")
        print(f"  Samples: {len(srcDf)}")
        print(f"  Low: {(srcDf['stressClass'] == 0).sum()}")
        print(f"  Moderate: {(srcDf['stressClass'] == 1).sum()}")
        print(f"  High: {(srcDf['stressClass'] == 2).sum()}")
    
    print("\nTraining model...")
    X = df[['sdnn', 'rmssd', 'meanHr', 'pnn50']].values
    y = df['stressClass'].values
    
    clf = StressClassifier()
    clf.train(X, y)
    clf.saveModel()
    
    print("\nDONE!")

Stress Classification - 3 Classes

Loading WESAD data...

Found 15 WESAD subjects
Processing S10...
Processing S11...
Processing S13...
Processing S14...
Processing S15...
Processing S16...
Processing S17...
Processing S2...
Processing S3...
Processing S4...
Processing S5...
Processing S6...
Processing S7...
Processing S8...
Processing S9...

WESAD data:
  Total samples: 1499
  Low: 981
  Moderate: 186
  High: 332
  SDNN range: 10.4 - 256.0
  RMSSD range: 3.6 - 271.4

Combining datasets...

Total samples: 1499

WESAD:
  Samples: 1499
  Low: 981
  Moderate: 186
  High: 332

Training model...

EVALUATION
              precision    recall  f1-score   support

         Low       0.85      0.67      0.75       196
    Moderate       0.21      0.41      0.28        37
        High       0.73      0.81      0.77        67

    accuracy                           0.67       300
   macro avg       0.60      0.63      0.60       300
weighted avg       0.74      0.67      0.70       300


Confusio

In [9]:
# main script

if __name__ == '__main__':
    print("Stress Classification - 3 Classes")
    print("="*50)
    
    samsungDir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\jsons'
    wesadDir = r'D:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2\HRVModule\XGBoost\WESAD'
    
    useSamsung = False
    useWesad = True
    
    allData = []
    

Stress Classification - 3 Classes


In [10]:
    if useSamsung:
        print("\nLoading Samsung data...")
        loader = SamsungDataLoader()
        stressData, hrvData = loader.loadAllSamsungFiles(samsungDir)
        
        if stressData and hrvData:
            dfStress = loader.processStressData(stressData)
            dfHrv = loader.processHrvData(hrvData)
            dfSamsung = loader.mergeStressAndHrv(dfStress, dfHrv)
            
            dfSamsung['source'] = 'Samsung'
            allData.append(dfSamsung[['sdnn', 'rmssd', 'stressClass', 'source']])
            print(f"Added {len(dfSamsung)} Samsung samples")
    


In [11]:
    if useWesad:
        print("\nLoading WESAD data...")
        wesadLoader = WesadDataLoader()
        dfWesad = wesadLoader.processWesadData(wesadDir)
        
        if dfWesad is not None and len(dfWesad) > 0:
            allData.append(dfWesad[['sdnn', 'rmssd', 'meanHr', 'pnn50', 'stressClass', 'source']])    
    print("\nCombining datasets...")
    df = pd.concat(allData, ignore_index=True)
    



Loading WESAD data...

Found 15 WESAD subjects
Processing S10...
Processing S11...
Processing S13...
Processing S14...
Processing S15...
Processing S16...
Processing S17...
Processing S2...
Processing S3...
Processing S4...
Processing S5...
Processing S6...
Processing S7...
Processing S8...
Processing S9...

WESAD data:
  Total samples: 1499
  Low: 981
  Moderate: 186
  High: 332
  SDNN range: 10.4 - 256.0
  RMSSD range: 3.6 - 271.4

Combining datasets...


In [12]:
    print(f"\nTotal samples: {len(df)}")
    for src in df['source'].unique():
        srcDf = df[df['source'] == src]
        print(f"\n{src}:")
        print(f"  Samples: {len(srcDf)}")
        print(f"  Low: {(srcDf['stressClass'] == 0).sum()}")
        print(f"  Moderate: {(srcDf['stressClass'] == 1).sum()}")
        print(f"  High: {(srcDf['stressClass'] == 2).sum()}")
    
    print("\nTraining model...")
    X = df[['sdnn', 'rmssd', 'meanHr', 'pnn50']].values
    y = df['stressClass'].values
    StressClassifier
    clf = ()
    clf.train(X, y)
    clf.saveModel()

# show where files saved
    print("\n" + "="*60)
    print("MODEL SAVED TO:")
    print("="*60)
    currentDir = os.getcwd()
    print(f"Directory: {currentDir}")
    print(f"Model: {os.path.join(currentDir, 'stress_model.pkl')}")
    print(f"Scaler: {os.path.join(currentDir, 'scaler.pkl')}")
    print("="*60)
    
    print("\nDONE!")


Total samples: 1499

WESAD:
  Samples: 1499
  Low: 981
  Moderate: 186
  High: 332

Training model...


AttributeError: 'tuple' object has no attribute 'train'

In [14]:
print("Looking for model files...")

# find model files
startPath = os.path.abspath('../../')
modelPath = None
scalerPath = None

for root, dirs, files in os.walk(startPath):
    if 'stress_model.pkl' in files:
        modelPath = os.path.join(root, 'stress_model.pkl')
        print(f"Found model at: {modelPath}")
    if 'scaler.pkl' in files:
        scalerPath = os.path.join(root, 'scaler.pkl')
        print(f"Found scaler at: {scalerPath}")

print("Loading model...")
model = joblib.load(modelPath)
print("Model loaded")

print("Loading scaler...")
scaler = joblib.load(scalerPath)
print("Scaler loaded")

print("All files loaded successfully")

# show file locations
print("\n" + "="*60)
print("MODEL FILE LOCATIONS:")
print("="*60)
print(f"Model file: {modelPath}")
print(f"Scaler file: {scalerPath}")
print("="*60 + "\n")

Looking for model files...
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\scaler.pkl
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\scaler.pkl
Loading model...
Model loaded
Loading scaler...
Scaler loaded
All files loaded successfully

MODEL FILE LOCATIONS:
Model file: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\stress_model.pkl
Scaler file: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\scaler.pkl



In [15]:
# load the saved model
predictor = StressPredictor()

# predict stress from HRV values
result = predictor.predictSingle(sdnn=45.2, rmssd=38.7, meanHr=75, pnn50=15)

print(result['stressLabel'])
print(result['confidence'])

Model loaded

Input: SDNN=45.20, RMSSD=38.70, MeanHR=75.00, PNN50=15.00
Predicted: LOW (confidence: 83.9%)
Show to user: YES
LOW
0.8385898470878601


In [16]:
# calculate from IBI intervals
ibi = np.array([850, 820, 830, 840, 825])  # your IBI data in ms

sdnn = np.std(ibi)
rmssd = np.sqrt(np.mean(np.diff(ibi) ** 2))
meanHr = 60000 / np.mean(ibi)
diffIbi = np.diff(ibi)
pnn50 = np.sum(np.abs(diffIbi) > 50) / len(diffIbi) * 100

result = predictor.predictSingle(sdnn, rmssd, meanHr, pnn50)


Input: SDNN=10.77, RMSSD=18.20, MeanHR=72.03, PNN50=0.00
Predicted: MODERATE (confidence: 54.4%)
Show to user: NO


In [18]:
predictor = StressPredictor(modelFile='stress_model.pkl', scalerFile='scaler.pkl')

# get new HRV data
sdnnValue = 50.5
rmssdValue = 42.3
meanHrValue = 72
pnn50Value = 20

prediction = predictor.predictSingle(sdnnValue, rmssdValue, meanHrValue, pnn50Value, showDetails=False)

if prediction['showToUser']:
    # display to user
    print(f"Stress: {prediction['stressLabel']}")
    print(f"Confidence: {prediction['confidence']:.0%}")

Model loaded
Stress: LOW
Confidence: 84%


In [20]:
predictor = StressPredictor()

# low stress example
predictor.predictSingle(sdnn=80, rmssd=65, meanHr=65, pnn50=30)

# moderate stress example
predictor.predictSingle(sdnn=48, rmssd=38, meanHr=80, pnn50=15)

# high stress example
predictor.predictSingle(sdnn=15, rmssd=10, meanHr=100, pnn50=5)


Model loaded

Input: SDNN=80.00, RMSSD=65.00, MeanHR=65.00, PNN50=30.00
Predicted: LOW (confidence: 67.9%)
Show to user: YES

Input: SDNN=48.00, RMSSD=38.00, MeanHR=80.00, PNN50=15.00
Predicted: HIGH (confidence: 87.8%)
Show to user: YES

Input: SDNN=15.00, RMSSD=10.00, MeanHR=100.00, PNN50=5.00
Predicted: LOW (confidence: 72.1%)
Show to user: YES


{'stressLevel': 0,
 'stressLabel': 'LOW',
 'confidence': 0.7210155129432678,
 'showToUser': True,
 'probabilities': {'low': 0.7210155129432678,
  'moderate': 0.009100452996790409,
  'high': 0.26988404989242554}}

In [22]:
predictor = StressPredictor()

# your watch gives you IBI data (in milliseconds)
ibiData = [850, 820, 830, 840, 825, 815, 835, 845, 828, 822]  # example IBI values

# calculate all 4 HRV features from IBI
ibi = np.array(ibiData)

sdnn = np.std(ibi)
rmssd = np.sqrt(np.mean(np.diff(ibi) ** 2))
meanHr = 60000 / np.mean(ibi)
diffIbi = np.diff(ibi)
pnn50 = np.sum(np.abs(diffIbi) > 50) / len(diffIbi) * 100

# predict
result = predictor.predictSingle(sdnn, rmssd, meanHr, pnn50)

if result['showToUser']:
    print(f"Stress: {result['stressLabel']}")
    print(f"Confidence: {result['confidence']:.0%}")

Model loaded

Input: SDNN=10.76, RMSSD=15.81, MeanHR=72.20, PNN50=0.00
Predicted: MODERATE (confidence: 58.3%)
Show to user: NO


In [24]:
model = clf.model
scaler = clf.scaler
print("Done")

AttributeError: 'tuple' object has no attribute 'model'

In [ ]:
# CELL 1: Imports

import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import schedule
import time
import os


# CELL 2: StressPredictor Class

class StressPredictor:
    
    def __init__(self, modelFile='stress_model.pkl', scalerFile='scaler.pkl', db=None):
        self.model = joblib.load(modelFile)
        self.scaler = joblib.load(scalerFile)
        self.db = db
        
        self.minConfidence = 0.60
        self.minSdnn = 10
        self.maxSdnn = 200
        self.minRmssd = 5
        self.maxRmssd = 150
        
        # demo user ID
        self.DEMO_USER_ID = "demo-user-123"
        self.DEMO_EMAIL = "demo@healmind.com"
        
        print("Model loaded")
    
    def predictSingle(self, sdnn, rmssd, meanHr, pnn50, showDetails=True):
        if not (self.minSdnn <= sdnn <= self.maxSdnn):
            return {'error': f'Invalid SDNN: {sdnn:.2f}'}
        if not (self.minRmssd <= rmssd <= self.maxRmssd):
            return {'error': f'Invalid RMSSD: {rmssd:.2f}'}
        
        X = np.array([[sdnn, rmssd, meanHr, pnn50]])
        xScaled = self.scaler.transform(X)
        
        pred = self.model.predict(xScaled)[0]
        proba = self.model.predict_proba(xScaled)[0]
        
        stressLabels = {0: 'LOW', 1: 'MODERATE', 2: 'HIGH'}
        confidence = proba.max()
        
        result = {
            'stressLevel': int(pred),
            'stressLabel': stressLabels[pred],
            'confidence': float(confidence),
            'showToUser': confidence >= self.minConfidence,
            'probabilities': {
                'class_0_low': float(proba[0]),
                'class_1_medium': float(proba[1]),
                'class_2_high': float(proba[2])
            }
        }
        
        if showDetails:
            print(f"\nInput: SDNN={sdnn:.2f}, RMSSD={rmssd:.2f}, MeanHR={meanHr:.2f}, PNN50={pnn50:.2f}")
            print(f"Predicted: {result['stressLabel']} (confidence: {confidence:.1%})")
            print(f"Show to user: {'YES' if result['showToUser'] else 'NO'}")
        
        return result
    
    def calculateHrvFromIbi(self, ibiData):
        if len(ibiData) < 2:
            return None
        
        ibi = np.array(ibiData)
        
        ibi = ibi[(ibi > 300) & (ibi < 2000)]
        
        if len(ibi) < 2:
            return None
        
        sdnn = np.std(ibi)
        rmssd = np.sqrt(np.mean(np.diff(ibi) ** 2))
        meanHr = 60000 / np.mean(ibi)
        diffIbi = np.diff(ibi)
        pnn50 = np.sum(np.abs(diffIbi) > 50) / len(diffIbi) * 100 if len(diffIbi) > 0 else 0
        
        return {
            'sdnn': sdnn,
            'rmssd': rmssd,
            'meanHr': meanHr,
            'pnn50': pnn50
        }
    
    def runBatch(self, hours=1):
        if self.db is None:
            print("No Firebase connection")
            return []
        
        print(f"\n{'='*60}")
        print(f"BATCH PROCESSING - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*60}")
        
        cutoff = datetime.now() - timedelta(hours=hours)
        print(f"Processing data from last {hours} hour(s)")
        print(f"Cutoff time: {cutoff.strftime('%Y-%m-%d %H:%M:%S')}")
        
        try:
            wearableRef = self.db.collection('wearable_data')
            
            docs = wearableRef.where('timestamp', '>=', cutoff).stream()
            
            processed = 0
            saved = 0
            errors = 0
            
            for doc in docs:
                try:
                    data = doc.data()
                    
                    if 'ibi' in data and isinstance(data['ibi'], list) and len(data['ibi']) > 0:
                        hrv = self.calculateHrvFromIbi(data['ibi'])
                        
                        if hrv is None:
                            errors += 1
                            continue
                        
                        prediction = self.predictSingle(
                            hrv['sdnn'], 
                            hrv['rmssd'], 
                            hrv['meanHr'], 
                            hrv['pnn50'],
                            showDetails=False
                        )
                    
                    elif all(k in data for k in ['sdnn', 'rmssd', 'meanHr', 'pnn50']):
                        prediction = self.predictSingle(
                            data['sdnn'],
                            data['rmssd'],
                            data['meanHr'],
                            data['pnn50'],
                            showDetails=False
                        )
                    
                    elif 'heart_rate' in data:
                        hr = data['heart_rate']
                        
                        if hr < 70:
                            sdnn, rmssd, pnn50 = 80, 65, 30
                        elif hr < 90:
                            sdnn, rmssd, pnn50 = 48, 38, 15
                        else:
                            sdnn, rmssd, pnn50 = 15, 10, 5
                        
                        prediction = self.predictSingle(
                            sdnn, rmssd, hr, pnn50,
                            showDetails=False
                        )
                    else:
                        errors += 1
                        continue
                    
                    if 'error' in prediction:
                        errors += 1
                        continue
                    
                    self.db.collection('stress_predictions').add({
                        'userId': self.DEMO_USER_ID,
                        'userEmail': self.DEMO_EMAIL,
                        'stress_probabilities': prediction['probabilities'],
                        'stressLevel': prediction['stressLevel'],
                        'stressLabel': prediction['stressLabel'],
                        'confidence': prediction['confidence'],
                        'prediction_timestamp': firestore.SERVER_TIMESTAMP,
                        'source': 'batch_prediction',
                        'source_doc_id': doc.id
                    })
                    
                    saved += 1
                    processed += 1
                    
                except Exception as e:
                    errors += 1
                    print(f"Error processing document {doc.id}: {e}")
                    continue
            
            print(f"\nBATCH RESULTS:")
            print(f"  Processed: {processed}")
            print(f"  Saved: {saved}")
            print(f"  Errors: {errors}")
            print(f"{'='*60}\n")
            
            return {
                'processed': processed,
                'saved': saved,
                'errors': errors,
                'timestamp': datetime.now()
            }
            
        except Exception as e:
            print(f"Batch processing failed: {e}")
            import traceback
            traceback.print_exc()
            return {'error': str(e)}
    
    def fetchAndPredictFromFirebase(self, hours=1):
        return self.runBatch(hours=hours)


# CELL 3: Find Firebase Credentials

print("Looking for Firebase credentials...")
startPath = os.path.abspath('../../')
credPath = None
for root, dirs, files in os.walk(startPath):
    if 'healmind-2025-firebase-adminsdk-fbsvc-12242dbda6.json' in files:
        credPath = os.path.join(root, 'healmind-2025-firebase-adminsdk-fbsvc-12242dbda6.json')
        break

print(f"Found at: {credPath}")


# CELL 4: Connect to Firebase

cred = credentials.Certificate(credPath)

try:
    firebase_admin.initialize_app(cred)
except:
    pass

db = firestore.client()
print("Firebase connected")


# CELL 5: Find Model Files

print("Looking for model files...")

startPath = os.path.abspath('../../')
modelPath = None
scalerPath = None

for root, dirs, files in os.walk(startPath):
    if 'stress_model.pkl' in files:
        modelPath = os.path.join(root, 'stress_model.pkl')
        print(f"Found model at: {modelPath}")
    if 'scaler.pkl' in files:
        scalerPath = os.path.join(root, 'scaler.pkl')
        print(f"Found scaler at: {scalerPath}")


# CELL 6: Load Model and Scaler

print("Loading model...")
model = joblib.load(modelPath)
print("Model loaded")

print("Loading scaler...")
scaler = joblib.load(scalerPath)
print("Scaler loaded")


# CELL 7: Initialize Predictor

print("Initializing predictor...")
predictor = StressPredictor(modelFile=modelPath, scalerFile=scalerPath, db=db)

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)


# CELL 8: Test Single Prediction (Optional)

# test the predictor
result = predictor.predictSingle(sdnn=48, rmssd=38, meanHr=80, pnn50=15)


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.9.18). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


Looking for Firebase credentials...
Found at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\healmind-2025-firebase-adminsdk-fbsvc-12242dbda6.json
Firebase connected
Looking for model files...
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\scaler.pkl
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\scaler.pkl
Loading model...
Model loaded
Loading scaler...
Scaler loaded
Initializing predictor...
Model loaded

SETUP COMPLETE

Input: SDNN=48.00, RMSSD=38.00, MeanHR=80.00, PNN50=15.00
Predicted: HIGH (confidence: 87.8%)
Show to user: YES


In [27]:
# CELL 8: Check What Data Exists in Firestore

print("Checking stress_predictions collection...\n")

# check total documents
allDocs = db.collection('stress_predictions').limit(10).get()

print("Sample documents:")
for doc in allDocs:
    data = doc.to_dict()
    timestamp = data.get('prediction_timestamp')
    userId = data.get('userId')
    
    if timestamp:
        if hasattr(timestamp, 'toDate'):
            dateStr = timestamp.toDate().strftime('%Y-%m-%d %H:%M:%S')
        else:
            dateStr = str(timestamp)
    else:
        dateStr = "No timestamp"
    
    print(f"  ID: {doc.id}")
    print(f"  Date: {dateStr}")
    print(f"  userId: {userId}")
    print(f"  Has probabilities: {'stress_probabilities' in data}")
    print()

# check date range
print("\nChecking date range of ALL data (without userId filter)...")
try:
    from datetime import datetime
    allDataQuery = db.collection('stress_predictions').order_by('prediction_timestamp').limit(1).get()
    for doc in allDataQuery:
        data = doc.to_dict()
        if data.get('prediction_timestamp'):
            earliest = data['prediction_timestamp'].toDate()
            print(f"  Earliest: {earliest.strftime('%Y-%m-%d %H:%M:%S')}")
    
    allDataQuery = db.collection('stress_predictions').order_by('prediction_timestamp', direction='DESCENDING').limit(1).get()
    for doc in allDataQuery:
        data = doc.to_dict()
        if data.get('prediction_timestamp'):
            latest = data['prediction_timestamp'].toDate()
            print(f"  Latest: {latest.strftime('%Y-%m-%d %H:%M:%S')}")
except Exception as e:
    print(f"  Error: {e}")

print("\n" + "="*60)


# CELL 9: Test Single Prediction (Optional)

# test the predictor
result = predictor.predictSingle(sdnn=48, rmssd=38, meanHr=80, pnn50=15)

Checking stress_predictions collection...

Sample documents:
  ID: 0lTIl2Lz58SpGIPHHugM
  Date: 2026-01-13 06:12:58.317000+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 0njjcR6Ne6nYOwWfVoFm
  Date: 2026-01-04 10:16:29.278012+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 0ys2x9j0yDEEmXTwcipo
  Date: 2026-01-11 15:55:34.853391+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 0zjdTmeITRJkOeiRFHhM
  Date: 2026-01-15 05:48:52.563846+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 163NEVgbKYWbAqp8UuC1
  Date: 2026-01-11 15:55:34.855390+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 1FIC7PVohQAXv4r1J2sm
  Date: 2026-01-15 02:33:14.014411+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

  ID: 1YTc4JLQDulPNaV6cxJc
  Date: 2026-01-04 10:16:31.791349+00:00
  userId: QOyZROlPzUf25tKPvv0FWnd3NZw2
  Has probabilities: True

 

In [29]:
print("Processing all data from the past week...")
print("This may take a while...\n")

# run batch on 7 days (168 hours) of data
result = predictor.runBatch(hours=168)

print("\nCOMPLETE!")
print(f"Total processed: {result.get('processed', 0)}")
print(f"Total saved: {result.get('saved', 0)}")
print(f"Total errors: {result.get('errors', 0)}")

Processing all data from the past week...
This may take a while...


BATCH PROCESSING - 2026-01-19 13:55:43
Processing data from last 168 hour(s)
Cutoff time: 2026-01-12 13:55:43

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


COMPLETE!
Total processed: 0
Total saved: 0
Total errors: 0


In [31]:

# CELL 9: Schedule and Run Batch

def job():
    predictor.runBatch(hours=1)


In [ ]:

schedule.every(1).minutes.do(job)

print("Scheduler started!")
print("Will run predictions every 5 minutes")
print("\nRunning first batch now...")

job()

print("\nKeep this cell running to continue processing...")

while True:
    schedule.run_pending()
    time.sleep(60)

Scheduler started!
Will run predictions every 5 minutes

Running first batch now...

BATCH PROCESSING - 2026-01-19 13:55:46
Processing data from last 1 hour(s)
Cutoff time: 2026-01-19 12:55:46

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


Keep this cell running to continue processing...

BATCH PROCESSING - 2026-01-19 13:56:46
Processing data from last 1 hour(s)
Cutoff time: 2026-01-19 12:56:46

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-19 13:57:47
Processing data from last 1 hour(s)
Cutoff time: 2026-01-19 12:57:47

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-19 13:58:47
Processing data from last 1 hour(s)
Cutoff time: 2026-01-19 12:58:47

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-19 13:59:47
Processing data from last 1 hour(s)
Cutoff time: 2026-01-19 12:59:47

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-19 14:00:47
Proces

Traceback (most recent call last):
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 169, in error_remapped_callable
    return _StreamingResponseIterator(
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 91, in __init__
    self._stored_first_result = next(self._wrapped)
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 538, in __next__
    return self._next()
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 962, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "Quota exceeded."
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B2404:6800:4001:818::200a%5D:443 {grpc_message:"Quota exceeded.", grpc_status:8}"
>

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
 


BATCH PROCESSING - 2026-01-20 02:35:28
Processing data from last 1 hour(s)
Cutoff time: 2026-01-20 01:35:28
Batch processing failed: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


Traceback (most recent call last):
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 169, in error_remapped_callable
    return _StreamingResponseIterator(
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 91, in __init__
    self._stored_first_result = next(self._wrapped)
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 538, in __next__
    return self._next()
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 962, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "Quota exceeded."
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B2404:6800:4001:818::200a%5D:443 {grpc_status:8, grpc_message:"Quota exceeded."}"
>

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
 


BATCH PROCESSING - 2026-01-20 02:40:48
Processing data from last 1 hour(s)
Cutoff time: 2026-01-20 01:40:48
Batch processing failed: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


...


BATCH PROCESSING - 2026-01-20 08:07:08
Processing data from last 1 hour(s)
Cutoff time: 2026-01-20 07:07:08
Batch processing failed: Timeout of 300.0s exceeded, last exception: 429 Quota exceeded.


Traceback (most recent call last):
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 169, in error_remapped_callable
    return _StreamingResponseIterator(
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 91, in __init__
    self._stored_first_result = next(self._wrapped)
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 538, in __next__
    return self._next()
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 962, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "Quota exceeded."
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B2404:6800:4001:818::200a%5D:443 {grpc_message:"Quota exceeded.", grpc_status:8}"
>

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
 


BATCH PROCESSING - 2026-01-20 08:12:35
Processing data from last 1 hour(s)
Cutoff time: 2026-01-20 07:12:35
Batch processing failed: Timeout of 300.0s exceeded, last exception: 503 IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)


Traceback (most recent call last):
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 169, in error_remapped_callable
    return _StreamingResponseIterator(
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\grpc_helpers.py", line 91, in __init__
    self._stored_first_result = next(self._wrapped)
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 538, in __next__
    return self._next()
  File "d:\laiba\anaconda3\envs\TF\lib\site-packages\grpc\_channel.py", line 962, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_status:14, grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the rem


BATCH PROCESSING - 2026-01-21 02:36:57
Processing data from last 1 hour(s)
Cutoff time: 2026-01-21 01:36:57

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-21 02:37:58
Processing data from last 1 hour(s)
Cutoff time: 2026-01-21 01:37:58

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0


BATCH PROCESSING - 2026-01-21 02:38:59
Processing data from last 1 hour(s)
Cutoff time: 2026-01-21 01:38:59

BATCH RESULTS:
  Processed: 0
  Saved: 0
  Errors: 0



In [20]:
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1.base_query import FieldFilter
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import schedule
import time
import os

# find firebase key
startPath = os.path.abspath('../../')
credPath = None
for root, dirs, files in os.walk(startPath):
    if 'healmind-2025-firebase-adminsdk-fbsvc-12242dbda6.json' in files:
        credPath = os.path.join(root, 'healmind-2025-firebase-adminsdk-fbsvc-12242dbda6.json')
        break

cred = credentials.Certificate(credPath)

try:
    firebase_admin.initialize_app(cred)
except:
    pass

db = firestore.client()
print("Firebase connected")

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


d:\laiba\anaconda3\envs\TF\lib\site-packages\google\api_core\_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.9.18). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


Firebase connected


In [21]:
print("Looking for model files...")

# find model files
startPath = os.path.abspath('../../')
modelPath = None
scalerPath = None

for root, dirs, files in os.walk(startPath):
    if 'stress_model.pkl' in files:
        modelPath = os.path.join(root, 'stress_model.pkl')
        print(f"Found model at: {modelPath}")
    if 'scaler.pkl' in files:
        scalerPath = os.path.join(root, 'scaler.pkl')
        print(f"Found scaler at: {scalerPath}")

print("Loading model...")
model = joblib.load(modelPath)
print("Model loaded")

print("Loading scaler...")
scaler = joblib.load(scalerPath)
print("Scaler loaded")

print("All files loaded successfully")

Looking for model files...
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\scaler.pkl
Found model at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\stress_model.pkl
Found scaler at: d:\laiba\Desktop\USM\CAT304W Drafts\Working\HealMind_ver2 - Copy\HealMind_Ver3\HRVModule\XG_Calc_predict\scaler.pkl
Loading model...
Model loaded
Loading scaler...
Scaler loaded
All files loaded successfully


In [22]:
def job():
    predictor.runBatch(hours=1)

schedule.every(5).minutes.do(job)

print("Scheduler started!")
print("Will run predictions every 5 minutes")
print("\nKeep this cell running to continue processing...")

job()

while True:
    schedule.run_pending()
    time.sleep(60)

Scheduler started!
Will run predictions every 5 minutes

Keep this cell running to continue processing...


AttributeError: 'StressPredictor' object has no attribute 'runBatch'